# Text preprocessing

Text is highly unstructured and needs to be prepared into a form that can be processed by machine learning algorithms. There are several different [approaches](https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction) for extracting features from text and we will explore a few of them in the next notebook. However, before extracting features from text, we need to "preprocess" it, i.e., "clean" and "standardize" it. This is because raw text can be "messy", especially when coming from social media platforms! We need to keep as many "informative" words as possible while discarding the "uninformative" ones. Removing unnecessary terms, i.e., the "noise", will improve our models' performance.

I will use the [Sentiment140](http://help.sentiment140.com/for-students/) public twitter corpus. This dataset contains ~500 tweets, labeled as positive, negative, or neutral. The dataset is available in the *data* folder.

In [1]:
# Import necessary libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os, re, random, string
from collections import defaultdict

# Gensim
import gensim
from gensim.models import Phrases

# Load helper functions
import tools

DATA_PATH = 'data'

In [2]:
# Load corpus (twitter dataset)

path = os.path.join(DATA_PATH,'sentiment_140.csv')
df   = pd.read_csv(path, header=None)

# Rename columns
df.columns = ['label','tweet']

# Re-order columns
df = df[['tweet','label']].copy()

# Drop duplicates
df = df.drop_duplicates('tweet')

# Remove empty rows
df = df.dropna()

# Rename labels
label_dict = {0:'neg', 2:'neutral', 4:'pos'}
df['label'] = df['label'].replace(label_dict)

# Reset index
df.reset_index(inplace=True,drop=True)
print('Data size: ',df.shape)

Data size:  (498, 2)


In [3]:
# Show a few samples

df.sample(5)

,tweet,label
397,"confirmed: it's Time Warner's fault, not Facebook's, that fb is taking about 3 minutes to load. so tempted to switch to verizon =/",neg
284,Heading to San Francisco,neutral
234,"@defsounds WTF is the point of deleting tweets if they can still be found in summize and searches? Twitter, please fix that. Thanks and bye",neg
38,RT @jessverr I love the nerdy Stanford human biology videos - makes me miss school. http://bit.ly/13t7NR,pos
80,SHOUT OUTS TO ALL EAST PALO ALTO FOR BEING IN THE BUILDIN KARIZMAKAZE 50CAL GTA! ALSO THANKS TO PROFITS OF DOOM UNIVERSAL HEMPZ CRACKA......,pos


There are many different ways for text preprocessing, depending on the application and the text's language. Below are a few issues that I would like to address in this particular dataset.

* remove URLs (e.g., http://bit.ly/19epAH, www.tinyurl.com/m595fk)
* remove RT (stands for retweet)
* remove Twitter usernames (e.g., @BlondeBroad)
* remove hashtags (e.g. #Adidas -> Adidas)
* remove punctuation. However, a few groupings, such as `:-), <3, : d`, etc., express emotion, so, depending on the task, we may want to keep them in the text.
* remove numbers (e.g. 2020, 2, 15, ...)
* perform case conversion (e.g. Good -> good, ...)
* remove stopwords
* remove non-ASCII characters
* standardize the number of repeated characters, e.g. ("I loooooooovvvvvveee" -> "I loovvee")
* expand contractions (e.g. "don't" -> "do not", "won't" -> "will not", ...)
* apply stemming
* ...

### Tokenization

Tokenization is the process of splitting text into **words**.

Example:

$$\text{I came to Bern by train.} → \text{[I, came, to, Bern, by, train.]}$$
  
  
**The *TweetTokenizer***

The *TweetTokenizer* is a nice tool from the NLTK library, specially designed for tokenizing tweets. Apart from spliting text into words, it offers a few additional key options:
- reduces the number of repeated characters within a token e.g. "everrrrr" -> "everrr" (use: *reduce_len=True*)
- removes Twitter usernames (use: *strip_handles=True*)
- preserves punctuation and emoticons.

In [4]:
# Import tokenizer
from nltk.tokenize import TweetTokenizer

# Create an instance of the tokenizer
tokenizer = TweetTokenizer(reduce_len=True, strip_handles=True)

In [5]:
# Example using emoticons and punctuation

sample_1 = "This is a cooool #dummysmiley: :-) :-P <3 and some arrows < > -> <--"

print(sample_1,'\n')
print(tokenizer.tokenize(sample_1))

This is a cooool #dummysmiley: :-) :-P <3 and some arrows < > -> <-- 

['This', 'is', 'a', 'coool', '#dummysmiley', ':', ':-)', ':-P', '<3', 'and', 'some', 'arrows', '<', '>', '->', '<--']


In [6]:
# Example using strip_handles and reduce_len parameters

sample_2 = '@remy: This is waaaaayyyy too much for you!!!!!!'

print(sample_2,'\n')
print(tokenizer.tokenize(sample_2))

@remy: This is waaaaayyyy too much for you!!!!!! 

[':', 'This', 'is', 'waaayyy', 'too', 'much', 'for', 'you', '!', '!', '!']


### Case folding

We usually convert all documents to lowercase. This is because we want our models to count e.g. "I" together with "i", "The" together with "the", etc.

In [7]:
sample_3 = "I don't like this movie!"

print('Original tweet: \t\t\t{}\n'.format(sample_3))

# Case folding
print('Convert to lowercase: \t\t\t{}\n'.format(sample_3.lower()))

Original tweet: 			I don't like this movie!

Convert to lowercase: 			i don't like this movie!



### Stopwords

Stopwords are words that are essential for a sentence to make sense, such as: "I", "the", "and", etc. The issue with stopwords is that they are: *very frequent* and *uninformative*. For most NLP applications, it is a good idea to remove them from text. 

Most NLP libraries provide pre-compiled lists of stopwords for several languages. In this notebook, we will use the list provided by the [NLTK library](https://www.nltk.org/). 

In [8]:
# Load english stopwords from nltk

import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords          
stopwords_english = stopwords.words('english')

[nltk_data] Downloading package stopwords to /Users/xydi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
# Print stopwords

print('{} stopwords in NLTK\'s list.\n'.format(len(stopwords_english)))
print(stopwords_english)

179 stopwords in NLTK's list.

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only',

Interestingly, the NLTK stopword list includes negation words such as:

- no, nor, not
- don't, didn't, wouldn't


In [10]:
# Check if 'not' is in stopwords_english

'not' in stopwords_english

True

In [11]:
# Check if "don't" is in stopwords_english

"don't" in stopwords_english

True

This may be an important point to keep in mind, depending on the application. For example, if we use the NLTK stopword list "out-of-the-box", then a sentence like:

$$\text{"I don't like this movie"}$$

will become:
$$\text{"like movie"}$$ 

We see that the processed sentence conveys the exact opposite sentiment from the original one! 

I will "customize" the NLTK stopword list to ensure that I don't remove negation words from tweets.

In [12]:
# Remove negation words from nltk's stopword list

not_stopwords = {'no', 'nor', 'not'} 
custom_stopwords = set([word for word in stopwords_english if word not in not_stopwords])

In [13]:
print(custom_stopwords)

{'these', 'doesn', 'ourselves', 'some', "hasn't", 'isn', 'any', 'further', 'few', "it's", 'there', 'll', 'shouldn', 'before', 'should', 'you', 'now', 'hadn', 'won', 's', "aren't", 'too', 'through', 'about', 'such', 'for', 'than', 've', 'the', 'its', 'after', 'o', 'will', 'only', 'me', 'being', 'on', 'd', 'him', 'didn', 'who', 'this', 'y', 'himself', 'from', "don't", 'they', 'their', 're', 'are', 'them', 'have', "mightn't", "won't", 'here', 'myself', 'in', 'until', 'hasn', 'haven', 'up', "you'd", 'were', "you've", 'off', 'been', 'over', 'wouldn', 'which', 'weren', 'yours', 'couldn', 'aren', 'to', 'i', 'she', 'did', 'into', "haven't", 'once', "she's", 'of', 'we', 'whom', 'between', 'ain', "hadn't", 'above', 'own', 'same', 'has', 'don', 'so', 'then', 'yourselves', 'or', 'with', 'where', 't', 'themselves', "couldn't", 'very', 'was', 'had', 'by', 'both', 'during', 'if', 'more', 'am', 'doing', 'other', 'but', 'can', 'herself', 'out', 'how', 'it', 'under', 'mustn', 'he', 'ma', 'my', 'and', "s

In [14]:
# Check if 'not' is in custom_stopwords

'no' in custom_stopwords

False

### Contractions

In practice, customizing the NLTK stopword lists will not be enough for preserving negation words in the text. We should also expand contractions: e.g. "don't" -> "do not". We can do this with the library "contractions".

$$\text{"I don't like this movie"}$$

In [15]:
import contractions

sample_3 = "I don't like this movie!"
print('Original tweet: \t\t\t{}\n'.format(sample_3))

# Case folding
print('Convert to lowercase: \t\t\t{}\n'.format(sample_3.lower()))

# Expand contractions
sample_3_expanded = contractions.fix(sample_3.lower())
print('Expand contractions: \t\t\t{}\n'.format(sample_3_expanded))

# Tokenize tweet
print('Tokenize: \t\t\t\t{}\n'.format(sample_3_expanded.split()))

# Remove stopwords
print('Remove stopwords: \t\t\t{}\n'.format([w for w in sample_3_expanded.split() if w not in custom_stopwords]))

Original tweet: 			I don't like this movie!

Convert to lowercase: 			i don't like this movie!

Expand contractions: 			i do not like this movie!

Tokenize: 				['i', 'do', 'not', 'like', 'this', 'movie!']

Remove stopwords: 			['not', 'like', 'movie!']



I made sure that the "important" words for guessing the sentiment of this tweet (i.e. "not" and "like") were preserved!

### Stemming

Another part of text normalization is stemming, in which we mainly strip suffixes from the end of the word. The Porter stemmer is a widely used stemming tool for the English language. Stemming helps to connect words and reduce the size of the vocabulary (i.e., the number of unique words in a corpus). However, it can produce non-words, i.e., words that you won't find in a dictionary.

In [16]:
# Load stemmer

from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()

In [17]:
# Stem of "retrieve"

stemmer.stem('retrieve')

'retriev'

In [18]:
# Stem of "retrieval"

stemmer.stem('retrieval')

'retriev'

In [19]:
# Stem of "retrieved"

stemmer.stem('retrieved')

'retriev'

### The process_tweet function

Now let's bring everything together and create the `process_tweet` funchion which takes a tweet as an argument and  preprocesses it.

In [20]:
# Create process_tweet function

def process_tweet(tweet):
    
    '''
    Preprocess raw samples of tweets.
    
    INPUT: 
    - tweet: raw text (string)
    
    OUTPUT:
    - processed_tweet: processed tweet (string)
    '''
    
    # Remove RT
    clean_tweet = re.sub(r'RT','',tweet)

    # Remove URL
    clean_tweet = re.sub(r'https?:\/\/[^\s]+','',clean_tweet)

    # Remove hash #
    clean_tweet = re.sub(r'#','',clean_tweet)

    # Remove numbers
    clean_tweet = re.sub(r'\d+','',clean_tweet)

    # Convert to lowercase
    clean_tweet = clean_tweet.lower()
    
    # Remove punctuation repetions (that are not removed by TweetTokenizer)
    clean_tweet = re.sub(r'([._:]){2,}','',clean_tweet)
    
    # Remove non-ascii chars
    clean_tweet = ''.join([c for c in str(clean_tweet) if ord(c) < 128])

    # Expand contractions
    clean_tweet = contractions.fix(clean_tweet)
    
    # Tokenize tweet
    tokens = tokenizer.tokenize(clean_tweet)

    # Remove punctuation (except emoticons), stopwords, single-char words and apply stemming
    clean_tokens = [stemmer.stem(w) for w in tokens if (w not in string.punctuation) and
                       (w not in custom_stopwords) and (len(w)>1)]
    
    # The stemmer strips the final 's but leaves the apostroph: warner's -> warner'
    # Here, I'm removing the apostroph from the end of words
    clean_tokens = [tok if tok[-1] != "'" else tok[:-1] for tok in clean_tokens]

    # Join tokens in a single string to recreate the tweet
    processed_tweet = ' '.join([tok for tok in clean_tokens])
    processed_tweet = processed_tweet.strip()
       
    return processed_tweet

Let's test this function with a few examples.

In [21]:
# Sample a tweet randomly from the corpus

tweet =  df.iloc[random.randint(0,len(df)-1),0]

print('Before cleaning: \t{}\n'.format(tweet))
print('After cleaning: \t{}\n'.format(process_tweet(tweet)))


Before cleaning: 	luke and i got stopped walking out of safeway and asked to empty our pockets and lift our shirts. how jacked up is that?

After cleaning: 	luke got stop walk safeway ask empti pocket lift shirt jack



Let's now go ahead and process all tweets in the corpus using the `process_tweet` function.

In [22]:
# Process all tweets (and add to list)

processed_tweets = [process_tweet(tweet).split() for tweet in df['tweet']]

In [23]:
# Show a few samples of processed tweets

processed_tweets[0:3]

[['looovvvee', 'kindl', 'not', 'dx', 'cool', 'fantast', 'right'],
 ['read', 'kindl', 'love', 'lee', 'child', 'good', 'read'],
 ['ok', 'first', 'asses', 'kindl', 'fuck', 'rock']]

### Collocations

There is one last thing we can do before moving on to feature extraction.

We can look for combinations of words that frequently appear together, such as: "Night at the museum", "Star Trek", "San Francisco", "North Korea", etc, and replace them by a unique token. 

Example: 

$$\text{"Star Trek" → "Star_Trek"}$$
$$\text{"San Francisco" → "San_Francisco"}$$
$$\text{"North Korea" → "North_Korea"}$$


We often call these *phrases* or *collocations*; they are word combinations that are more common in the corpus than the individual words themselves. (*Note: "that is" is not considered a collocation*).

I will use Gensim's `models.phrases` to detect phrases (collocations) in our corpus. "Phrases" will identify the most common collocations and join the constituent tokens into a single token, using the "_" glue character. 

*Documentation*
* Gensim's website: https://radimrehurek.com/gensim/models/phrases.html
* Mikolov, *et. al*: ["Distributed Representations of Words and Phrases and their Compositionality"](https://arxiv.org/pdf/1310.4546.pdf)

In [24]:
# Detect collocations in corpus

for i in ['bigrams']:
    print('Computing collocations ({})...'.format(i))
    
    bigram = gensim.models.Phrases(processed_tweets,   # Expected format: list of tokenized documents
                                   min_count=3,        # Ignore all words and bigrams with total collected count lower than this value.
                                   delimiter=b'_')     # Glue character used to join collocation tokens

    bigram_model = gensim.models.phrases.Phraser(bigram)

    # Add detected collocations to corpus
    processed_tweets = [' '.join(bigram_model[processed_tweet]) for processed_tweet in processed_tweets]
    print('Done!')
    
# Add processed tweets to dataframe
df['processed_tweet'] = processed_tweets

Computing collocations (bigrams)...
Done!


In [25]:
# Remove empty tweets
df = df[df['processed_tweet'].apply(len) != 0].copy()

# Reindex dataframe
df.reset_index(inplace=True,drop=True)

# Re-order columns
df = df[['tweet','processed_tweet','label']]

print('Data size: ',df.shape)

Data size:  (498, 3)


This is how the processed tweets look like: 

In [26]:
df.sample(5)

,tweet,processed_tweet,label
429,still hungry after eating....,still hungri eat,neutral
99,"It's a bank holiday, yet I'm only out of work now. Exam season sucks:(",bank holiday yet work exam season suck :(,neg
415,Portland city politics may undo baseball park http://tinyurl.com/lpjquj,portland citi polit may undo basebal park,neutral
295,I seriously underestimated Malcolm Gladwell. I want to meet this dude.,serious underestim malcolm_gladwel want meet dude,pos
169,Going to the dentist later.:|,go dentist later,neg


### Manually label tweets into topics

For later analysis, I will now label tweets manually based on selected keywords.

For example, we will add any tweet mentioning movie-related terms (e.g. "Night at the museum", "Star Trek", "movie", etc.) to the category: "movies". Tweets mentioning politics-related terms (e.g. "north_korea", "obama", "pelosi", "bush", "china", "india", "iran", etc.) will be added to "politics". Tweets mentionning sports-related terms (e.g. "lebron", "laker", "basebal", "basketbal", "fifa", "ncaa", "roger", "feder", etc.) will be added to "sports". And so on.

We will use the function `add_to_topic` to label tweets manually.

In [27]:
# Call function add_to_topic() 

df = tools.add_to_topic(df)

In [28]:
# Show a few samples

df.sample(5)

,tweet,processed_tweet,label,topic
223,#at&amp;t is complete fail.,complet fail,neg,unlabeled
132,"Night at the Museum, Wolverine and junk food - perfect monday!",night_museum wolverin junk food perfect monday,pos,food
441,DevSnippets : jQuery Tools - Javascript UI Components for the Web... http://inblogs.org/go/hfuqt,devsnippet jqueri tool javascript ui compon web,neutral,IT
97,"I hate revision, it's so boring! I am totally unprepared for my exam tomorrow :( Things are not looking good...",hate revis bore total unprepar exam tomorrow :( thing not look good,neg,unlabeled
306,I'm really loving the new search site Wolfram/Alpha. Makes Google seem so ... quaint. http://www72.wolframalpha.com/,realli love new search site wolfram alpha make googl seem quaint,pos,IT


In [29]:
# Export data

#df.to_csv('clean_sentiment_140.csv',index=False)